In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from datetime import datetime as dt
import torch
print(torch.__version__)
print(torch.cuda.is_available())

import torch.nn as nn
import torch.nn.functional as f
from torch import optim

import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import transforms

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from attacks import fgsm, pgd
from model import SmallIntervalNet


1.6.0
False


In [3]:
from datetime import datetime as dt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def clean_acc(model, loader):
    clean_correct = 0.
    model.eval()
    with torch.no_grad():
        for k, (X, y) in enumerate(loader, 1):
            start = dt.now().replace(microsecond=0)
            X, y = X.to(device), y.to(device)
            l, _ = model(torch.cat([X, X], dim=1))
            clean_correct += (l.argmax(dim=1) == y).sum().item()
            end = dt.now().replace(microsecond=0)
            print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
        print(" " * 50, end="\r")
        acc = round(clean_correct / len(loader.dataset), 4)
        print(f"clean accuracy: {acc}")
    return acc

def adv_acc(model, loader, loss_fn, attack, attack_params):
    model.eval()
    adv_correct = [0] * len(attack_params)
    for i, (name, params) in enumerate(attack_params):
        for k, (X, y) in enumerate(loader, 1):
            start = dt.now().replace(microsecond=0)
            X, y = X.to(device), y.to(device)
            noise = attack(model, X, y, loss_fn, **params)
            l, _ = model(torch.cat([X+noise, X+noise], dim=1))
            adv_correct[i] += (l.argmax(dim=1) == y).sum().item()
            end = dt.now().replace(microsecond=0)
            print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
        print(" " * 50, end="\r")
        print(f"name: {name} accuracy: {(adv_correct[i] / len(loader.dataset)):.4f}")
    return [round(a/len(loader.dataset), 4) for a in adv_correct]



In [4]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

ds = MNIST('../data', train=False, target_transform=None, download=True, transform=test_transform)
test_loader = DataLoader(ds, 50, shuffle=True)

In [5]:
attack1 = {
    "epsilon": 8/255,
    "alpha": 2/255,
    "num_iter": 20
}

attack2 = {
    "epsilon": 8/255,
    "alpha": 2/255,
    "num_iter": 40
}

attack3 = {
    "epsilon": 0.1,
    "alpha": 2/255,
    "num_iter": 10
}

attack4 = {
    "epsilon": 0.1,
    "alpha": 2/255,
    "num_iter": 20
}

attack1 = ("eps=8/255,a=2,i=20", attack1)
attack2 = ("eps=8/255,a=2,i=40", attack2)
attack3 = ("eps=0.1,a=2,i=10", attack3)
attack4 = ("eps=0.1,a=2,i=20", attack4)

params20 = [attack1, attack3]
params40 = [attack3, attack4]


In [6]:
model = SmallIntervalNet().to(device)
model.load_state_dict(torch.load("saved/model_git.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(),
                  pgd, [attack1, attack2, attack3, attack4])




clean accuracy: 0.9608                            
name: eps=8/255,a=2,i=20 accuracy: 0.9446         
name: eps=8/255,a=2,i=40 accuracy: 0.9446         
name: eps=0.1,a=2,i=10 accuracy: 0.9042           
name: eps=0.1,a=2,i=20 accuracy: 0.8462           
